In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# ----------------------------
# Step 1: Preprocess the dataset
# ----------------------------

# Separate features and labels
features = dataset.drop("expenses", axis=1)
labels = dataset["expenses"]

# Identify categorical and numerical columns
categorical_features = ["sex", "smoker", "region"]
numerical_features = ["age", "bmi", "children"]

# Split into train and test sets (80/20 split)
from sklearn.model_selection import train_test_split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Preprocessing: OneHotEncode categorical, scale numerical
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), categorical_features)
    ]
)

# Fit on training data and transform both train/test sets
train_dataset = preprocessor.fit_transform(train_dataset)
test_dataset = preprocessor.transform(test_dataset)

# ----------------------------
# Step 2: Build the regression model
# ----------------------------
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(train_dataset.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Single output for regression
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mean_absolute_error', 'mse']
)

model.summary()

# ----------------------------
# Step 3: Train the model
# ----------------------------
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
